Solución Taller 5 - TF-IDF

In [3]:
# Librerias

import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Punto 1: Leer el archivo Princesas.csv usando pandas y crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [4]:
datos = pd.read_csv ('C:/Users/Nadia/Documents/Especialización/NPL/Clases y Talleres/Princesas.csv')

In [5]:
print(datos)

       Princesa                                       Personalidad
0  Blancanieves  Blancanieves es una princesa de noble cuna que...
1    Cenicienta  Cenicienta es inicialmente una sirvienta en su...
2        Aurora  La Princesa Aurora, la Bella Durmiente, es la ...
3         Bella  Bella es una muchacha que vive en la campiña f...
4        Jasmín  Cuando se introdujo por primera vez, la Prince...
5    Pocahontas  El nombre de Pocahontas significa "Pequeña Sil...
6         Mulan  Mulan es atípica a los anteriores papeles feme...
7         Tiana  Es una joven camarera que sueña con ser dueña ...
8        Mérida  Mérida llama la atención por su característico...
9         Moana  Moana, una joven de 16 años de edad, hija únic...


In [6]:

def pre_procesada(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords_sp]
    texto = [spanishStemmer.stem(palabra) for palabra in texto]
    texto = " ".join(texto)
    return texto

In [7]:
datos['pre_procesada'] = datos['Personalidad'].apply(lambda texto:pre_procesada(texto))

In [8]:
datos

,Princesa,Personalidad,pre_procesada
0,Blancanieves,Blancanieves es una princesa de noble cuna que...,blancaniev princes nobl cun ve forz servidumbr...
1,Cenicienta,Cenicienta es inicialmente una sirvienta en su...,cenicient inicial sirvient cas constant objet ...
2,Aurora,"La Princesa Aurora, la Bella Durmiente, es la ...",princes auror bell durmient hij unic rein flor...
3,Bella,Bella es una muchacha que vive en la campiña f...,bell muchach viv campiñ frances padr inventor ...
4,Jasmín,"Cuando se introdujo por primera vez, la Prince...",introduj primer vez princes jasmin poc dias de...
5,Pocahontas,"El nombre de Pocahontas significa ""Pequeña Sil...",nombr pocahont signif pequeñ silenci bas figur...
6,Mulan,Mulan es atípica a los anteriores papeles feme...,mul atip anterior papel femenin pelicul disney...
7,Tiana,Es una joven camarera que sueña con ser dueña ...,jov camarer sueñ ser dueñ propi restaur algun ...
8,Mérida,Mérida llama la atención por su característico...,mer llam atencion caracterist pel anaranj oscu...
9,Moana,"Moana, una joven de 16 años de edad, hija únic...",moan jov años edad hij unic sucesor import jef...


Punto 2:  Crear la matriz TF-IDF

In [29]:
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(datos['Princesa'].values)

tfidf_matrix = pd.DataFrame(tfidf.toarray(), columns=tfidf_vect.get_feature_names())
tfidf_matrix.index = datos.index

tfidf_matrix=tfidf_matrix.round(3)
tfidf_matrix


,aurora,bella,blancanieves,cenicienta,jasmín,moana,mulan,mérida,pocahontas,tiana
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0



Punto 3 - Distancia del Coseno

1. Calcular la distancia del coseno entre cada una de las princesas


In [27]:
from sklearn.metrics.pairwise import cosine_distances

dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos

,aurora,bella,blancanieves,cenicienta,jasmín,moana,mulan,mérida,pocahontas,tiana
aurora,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
bella,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
blancanieves,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
cenicienta,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
jasmín,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
moana,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
mulan,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
mérida,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
pocahontas,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
tiana,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


2. ¿Cuáles son las princesas más parecidas?, ¿Cuáles son las princesas más diferentes?
Debibo a que no me tomo la acción de ampliar la cantidad de decimales no pude observar con facilidad cuale princesas presentaban diferencia o igualdad.
